### Payout when taking random action each round

In [127]:
import gym
env = gym.make('Blackjack-v0')
env.reset()

num_rounds = 1000
num_samples = 1000
total_payout = 0

for _ in range(num_samples):
    round = 1
    while round <= num_rounds:
        action = env.action_space.sample()   
        # print (env._get_obs())
        # print (action)
        obs, payout, is_done, _ = env.step(action)
        # print (obs, payout, is_done)
        total_payout += payout
        if is_done:
            env.reset()
            round += 1
    
print ("Average payout after {} rounds is {}".format(num_rounds, total_payout/num_samples))

[2017-08-04 00:37:10,515] Making new env: Blackjack-v0


Average payout after 1000 rounds is -396.557


### Payout when using normal play strategy
Strategy from figure 11 [here](https://pdfs.semanticscholar.org/e1dd/06616e2d18179da7a3643cb3faab95222c8b.pdf)

In [126]:
def normal_strategy(player_sum, dealer_up_card):
    """
    2D array storing actions
    Each row corresponds to player sum- from 2 to 21
    Each column corresponds to dealer_up_card- from 1 to 10
    """
    actions = [[1]*10]*8 # 2 to 9
    actions.append([1]*4 + [0]*2 + [1]*4) #10
    actions.append([1] + [0]*6 + [1]*3) #11
    actions.append([0] + [1]*9) #12
    actions.append([0]*2 + [1]*8) #13
    actions.append([0]*1 + [1]*9) #14
    actions.append([0]*2 + [1]*8) #15
    actions.append([0]*5 + [1]*5) #16
    actions.append([0]*4 + [1]*6) #17
    actions.extend([[0]*10]*4) # 18 to 21
    
    # dealer_up_card-2 takes care of input 1 which correcly looks up last column
    return actions[player_sum-2][dealer_up_card-2]

assert (normal_strategy(15, 2)) == 0
assert (normal_strategy(15, 1)) == 1


num_rounds = 1000
num_samples = 1000
total_payout = 0

for _ in range(num_samples):
    round = 1
    while round <= num_rounds:
        player_sum, dealer_up_card, is_done = (env._get_obs())
        action = normal_strategy(player_sum, dealer_up_card)
        # print (env._get_obs())
        # print (action)
        obs, payout, is_done, _ = env.step(action)
        # print (obs, payout, is_done)
        total_payout += payout
        if is_done:
            env.reset()
            round += 1
    
print ("Average payout after {} rounds is {}".format(num_rounds, total_payout/num_samples))


Average payout after 1000 rounds is -100.769
